In [18]:
import pandas as pd
import pickle
import string
import random
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score

In [19]:
eng_stopwords = stopwords.words('english')
punctuation_list = string.punctuation
wnl = WordNetLemmatizer()

def remove_stopwords(w_list):
     return [word for word in w_list if word not in eng_stopwords]

def remove_punctuation(w_list):
     return [word for word in w_list if word not in punctuation_list]

def remove_number(w_list):
     return [word for word in w_list if word.isalpha()]

def get_pos_tag(tag):
     if tag == 'jj':
          return 'a'
     elif tag in ['nn', 'rb', 'vb']:
          return tag[0]
     else:
          return None

def lemmatizing_words(w_list):
     lemmatized = []
     tagging = pos_tag(w_list)
     for word, tag in tagging:
          label = get_pos_tag(tag)
          if label != None:
               lemmatized.append(wnl.lemmatize(word, label))
          else:
               lemmatized.append(wnl.lemmatize(word))
     return lemmatized

In [20]:
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    words = word_tokenize(text)
    words = remove_stopwords(words)
    words = remove_punctuation(words)
    words = remove_number(words)
    words = lemmatizing_words(words)
    return ' '.join(words)

In [21]:
def training_model():
     dataset = pd.read_csv('./dataset_rm.csv')
     dataset = dataset.dropna(subset=['tweet', 'label'])

     comments_list = dataset['tweet'].apply(preprocess_text).tolist()
     label_list = dataset['label'].tolist()

     vectorizer = TfidfVectorizer()
     word_tfidf = vectorizer.fit_transform(comments_list)

     X_train, X_test, y_train, y_test = train_test_split(word_tfidf, label_list, test_size=0.2, random_state=42)
     random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
     random_forest.fit(X_train, y_train)

     y_pred = random_forest.predict(X_test)

     accuracy = accuracy_score(y_test, y_pred)
     report = classification_report(y_test, y_pred)
     precision = precision_score(y_test, y_pred, pos_label=1)
     recall = recall_score(y_test, y_pred, pos_label=1)
     f1 = f1_score(y_test, y_pred, pos_label=1)

     print("Accuracy:", accuracy)
     print("Classification Report:\n", report)
     print(f'Precision: {precision:.2f}')
     print(f'Recall: {recall:.2f}')
     print(f'F1 Score: {f1:.2f}')

     file = open('random_forest.pickle','wb')
     pickle.dump(random_forest, file)
     file.close()

     file = open('tfidf_random_forest.pickle','wb')
     pickle.dump(vectorizer, file)
     file.close()

     return random_forest, vectorizer
     

In [22]:
model, vectorizer = training_model()

Accuracy: 0.6
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.60      1.00      0.75        12

    accuracy                           0.60        20
   macro avg       0.30      0.50      0.37        20
weighted avg       0.36      0.60      0.45        20

Precision: 0.60
Recall: 1.00
F1 Score: 0.75


C:\Users\Daniel\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daniel\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Daniel\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [23]:
def classify_comment(comment, model, vectorizer):
    processed_comment = preprocess_text(comment)
    comment_tfidf = vectorizer.transform([processed_comment])
    prediction = model.predict(comment_tfidf)
    print(prediction)
    return prediction[0]

In [24]:
new_comment = "your face is very pretty"

result = classify_comment(new_comment, model, vectorizer)

print(f"The comment '{new_comment}' is classified as: {result}")

[1]
The comment 'your face is very pretty' is classified as: 1
